In [ ]:
#Import necessary modules
import matplotlib.pyplot as plt
import os
import numpy as np
import urllib.request
from PIL import Image

In [ ]:
# Image preprocessing - Resizing and normalizing the image
size = (128,128)
def preProcessImage(image):
    resizedImage = image.resize(size)
    imageArray = np.array(resizedImage)
    if imageArray.dtype == np.uint8:
        imageArray = imageArray.astype(np.float32) / 255.0
    return imageArray

In [ ]:
#JPG to array conversion
def jpgToArray(folderPath):
    imageList = []
    for filename in os.listdir(folderPath):
        if filename.endswith(".jpg"):
            imagePath = os.path.join(folderPath, filename)
            image = Image.open(imagePath)
            preProcessedImage = preProcessImage(image)
            imageList.append(preProcessedImage)
    return np.array(imageList)

In [ ]:
#Mount drive to colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
'''Load, concatenate, and shuffle images from fire and no-fire directories
Assign label 1 to images with fire and 0 to images with no fire'''
def loadAndLabelImages():
  fireImagePath = "/content/drive/MyDrive/Fire-Detection/1/"
  noFireImagePath = "/content/drive/MyDrive/Fire-Detection/0/"
  fireImageArray = jpgToArray(fireImagePath)
  noFireImageArray = jpgToArray(noFireImagePath)
  np.random.seed(1)
  combinedArray = np.concatenate((fireImageArray, noFireImageArray), axis=0)
  indices = np.random.permutation(len(combinedArray))
  combinedArray = combinedArray[indices]
  fireLabels = np.ones(len(fireImageArray))
  noFirelabels = np.zeros(len(noFireImageArray))
  combinedLabels = np.concatenate((fireLabels, noFirelabels), axis=0)
  combinedLabels = combinedLabels[indices]
  return combinedArray,combinedLabels

In [ ]:
#Split the data into train and test sets
def trainTestSplit():
  combined=loadAndLabelImages()
  combinedArray=combined[0]
  combinedLabels=combined[1]
  from sklearn.model_selection import train_test_split
  trainImages, testImages, trainLabels, testLabels = train_test_split(combinedArray, combinedLabels, test_size=0.4)
  return trainImages,trainLabels,testImages,testLabels

In [ ]:
trainTestData=trainTestSplit()
trainImages=trainTestData[0]
trainLabels=trainTestData[1]
testImages=trainTestData[2]
testLabels=trainTestData[3]

import tensorflow as tf
from tensorflow.keras import layers, models

#CNN model definition
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
#Compile the model
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [ ]:
#Train the model
r = model.fit(trainImages,trainLabels, validation_data = (testImages,testLabels), epochs = 20)

In [ ]:
# Function to predict fire in an online image
def predictFireOnline(image):
  imageArray=preProcessImage(image)
  preProcessedImage=np.expand_dims(imageArray, axis=0)
  if preProcessedImage is not None:
    prediction = model.predict(preProcessedImage)
    if prediction >= 0.5:
      return "Fire"
    else:
      return "Not Fire"
# Example usage
imageUrl="https://media.npr.org/assets/img/2022/07/28/gettyimages-1005969960_wide-1ca4677c6b5d9405eef93f46d735ff2ed96cee03.jpg?s=1100&c=50&f=jpeg"
#Replace example image URL with required image URL
imageData = urllib.request.urlopen(imageUrl)
image = Image.open(imageData)
plt.imshow(image)
plt.axis('off')
plt.show()
result = predictFireOnline(image)
print("Prediction:", result)
